In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import pickle
import random
from sklearn.model_selection import train_test_split

from google.colab.patches import cv2_imshow

In [ ]:
IMG_HEIGHT = 160
IMG_WIDTH = 160
epochs = 20
batch_size = 6
ImgDir = "/content/drive/MyDrive/Colab Notebooks/Bakteri/"

In [ ]:
features = os.listdir(f"{ImgDir}image/")
labels = os.listdir(f"{ImgDir}mask/")

print(len(features), len(labels))

123 123


In [ ]:
X = features
y = labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.15, random_state=1)

print(len(X_train), len(X_val), len(X_test))

86 31 6


In [ ]:
def keras_generator_train_val_test(batch_size, choice="train"):
    if choice == "train":
        X = X_train
        y = y_train
    elif choice == "val":
        X = X_val
        y = y_val
    elif choice == "test":
        X = X_test
        y = y_test
    else:
        print("Invalid Option")
        return False
        
    while True:
        x_batch = []
        y_batch = []

        for i in range(batch_size):
            x_rand = random.choice(X)
            y_rand = x_rand[:-4]+".jpg"
            # print(y_rand)
            x_path = f"{ImgDir}image/{x_rand}"
            y_path = f"{ImgDir}mask/{y_rand}"

            x = cv2.imread(x_path)
            x = cv2.resize(x, (IMG_WIDTH, IMG_HEIGHT))
            y = cv2.imread(y_path)
            y = cv2.resize(y, (IMG_WIDTH, IMG_HEIGHT))

            x = x / 255.0
            y = y / 255.0
            
            x_batch.append(x)
            y_batch.append(y)

        x_batch = np.array(x_batch)

        y_batch = {'seg': np.array(y_batch)}

        yield x_batch, y_batch

# for x, y in keras_generator_train_val_test(2, choice="train"):
#     break

# print(x.shape, y['seg'].shape)

In [ ]:
# train_data = keras_generator_train_val_test(batch_size, choice="train")
# validation_data = keras_generator_train_val_test(batch_size, choice="val")

In [ ]:
# print(train_data)
# print(validation_data)

In [ ]:
# cv2_imshow(x[0] * 255.)
# cv2_imshow(y['seg'][0] * 255.)

In [ ]:
def get_model():
    in1 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3 ))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])

    losses = {'seg': 'binary_crossentropy'}

    metrics = {'seg': ['acc']}
    model.compile(optimizer="adam", loss = losses, metrics=metrics)

    return model

In [ ]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, epoch, logs={}):

        res_dir = "/content/drive/MyDrive/Colab Notebooks/Bakteri/result_white_background"

        try:
            os.makedirs(res_dir)
        except:
            print(f"{res_dir} directory already exist")

        print('Training: epoch {} begins at {}'.format(epoch, datetime.datetime.now().time()))

    def on_epoch_end(self, epoch, logs=None):
        res_dir = "/content/drive/MyDrive/Colab Notebooks/Bakteri/result_white_background/"
        print('Training: epoch {} ends at {}'.format(epoch, datetime.datetime.now().time()))
        
        for x_test, y_test in keras_generator_train_val_test(batch_size, choice="test"):
            break
        p = np.reshape(x_test[0], (1, 160, 160, 3))
        prediction = self.model.predict(p)

        x_img = f"{res_dir}{epoch}_X_input.jpg"
        y_img = f"{res_dir}{epoch}_Y_truth.jpg"
        predicted_img = f"{res_dir}{epoch}_Y_predicted.jpg"

        cv2.imwrite(x_img, x_test[0] * 255.)
        cv2.imwrite(y_img, y_test['seg'][0] * 255.)
        cv2.imwrite(predicted_img, prediction[0] * 255.)

In [ ]:
tf.keras.backend.clear_session()
model = get_model()

In [ ]:
model.summary()

In [ ]:
model_name = "/content/drive/MyDrive/Colab Notebooks/model/"+"Unet_Bakteri_white_background.h5"

modelcheckpoint = ModelCheckpoint(model_name,
                                  monitor='val_loss',
                                  mode='auto',
                                  verbose=1,
                                  save_best_only=True)

lr_callback = ReduceLROnPlateau(min_lr=0.001)

callback_list = [modelcheckpoint, lr_callback, MyCustomCallback()]

history = model.fit(
    keras_generator_train_val_test(batch_size, choice="train"),
    validation_data = keras_generator_train_val_test(batch_size, choice="val"),
    validation_steps = 20,
    steps_per_epoch= 20,
    epochs=epochs,
    verbose=1, 
    shuffle=True,
    callbacks = callback_list,
)


/content/drive/MyDrive/Colab Notebooks/Bakteri/result_white_background directory already exist
Training: epoch {} begins at 13:31:04.753578
Epoch 1/20
20/20 [==============================] - 98s 5s/step - loss: 0.6159 - acc: 0.3855 - val_loss: 0.5043 - val_acc: 0.0601

Epoch 00001: val_loss improved from inf to 0.50426, saving model to /content/drive/MyDrive/Colab Notebooks/model/Unet_Bakteri_white_background.h5
Training: epoch 0 ends at 13:32:43.307531
Epoch 2/20
20/20 [==============================] - 97s 5s/step - loss: 0.4739 - acc: 0.2658 - val_loss: 0.3831 - val_acc: 0.2421

Epoch 00002: val_loss improved from 0.50426 to 0.38309, saving model to /content/drive/MyDrive/Colab Notebooks/model/Unet_Bakteri_white_background.h5
Training: epoch 1 ends at 13:34:23.288733
Epoch 3/20
20/20 [==============================] - 97s 5s/step - loss: 0.3790 - acc: 0.3000 - val_loss: 0.3175 - val_acc: 0.3863

Epoch 00003: val_loss improved from 0.38309 to 0.31747, saving model to /content/drive/

In [ ]:
res_dir = "/content/drive/MyDrive/Colab Notebooks/Bakteri/hasil_tes/"

X = X_test
Y = y_test

for a in range(len(X)):
  print(a)

  x_rand = X[a]
  y_rand = x_rand[:-4]+".jpg"

  x_path = f"{ImgDir}image/{x_rand}"
  y_path = f"{ImgDir}mask/{y_rand}"

  x = cv2.imread(x_path)
  x = cv2.resize(x, (IMG_WIDTH, IMG_HEIGHT))
  y = cv2.imread(y_path)
  y = cv2.resize(y, (IMG_WIDTH, IMG_HEIGHT))

  x = x / 255.0
  y = y / 255.0

  p = np.reshape(x, (1, 160, 160, 3))
  prediction = model.predict(p)

  x_img = f"{res_dir}{a}_X_input.jpg"
  y_img = f"{res_dir}{a}_Y_truth.jpg"
  predicted_img = f"{res_dir}{a}_Y_predicted.jpg"

  cv2.imwrite(x_img, x * 255.)
  cv2.imwrite(y_img, y * 255.)
  cv2.imwrite(predicted_img, prediction[0] * 255.)